In [12]:
import pandas as pd
import pyodbc
import re
import os

os.makedirs("logs", exist_ok=True)
LOG_FILE = "logs/etl_error_log.txt"

def log_error(message):
    """Write error messages to log file."""
    with open(LOG_FILE, "a") as log:
        log.write(message + "\n")

try:
    try:
        sales_data = pd.read_json('C:/Users/muham/Downloads/MidLevel_DataEngineer_Assessment/sales_data.json')
        customer_data = pd.read_json('C:/Users/muham/Downloads/MidLevel_DataEngineer_Assessment/customer_data.json')
    except Exception as e:
        log_error(f"File Read Error: {str(e)}")
        raise

    try:
        customer_data = customer_data.dropna(subset=['customer_id'])
        customer_data['join_date'] = pd.to_datetime(customer_data['join_date'], errors='coerce')
        customer_data['loyalty_points'] = customer_data['loyalty_points'].fillna(0)

        sales_data['date'] = pd.to_datetime(sales_data['date'], errors='coerce')
        sales_data['total_price'] = sales_data['quantity'] * sales_data['product'].apply(lambda x: x['price'])

        products_list = []
        for _, row in sales_data.iterrows():
            product = row['product']
            products_list.append({
                'product_id': product['id'],
                'product_name': product['name'],
                'product_category': product['category'],
                'price': product['price']
            })

        products_data = pd.DataFrame(products_list).drop_duplicates(subset=['product_id'])

        def clean_name(name):
            try:
                name = ''.join(e for e in name if e.isalnum() or e.isspace())
                if len(name) > 100:
                    name = name[:100]
                if not name:
                    name = "Unknown Name"
                return name
            except Exception as e:
                log_error(f"Name Cleaning Error: {str(e)}")
                return "Unknown Name"

        def clean_email(email):
            try:
                if pd.isna(email) or '@' not in email:
                    return "invalid_email@example.com"
                return email
            except Exception as e:
                log_error(f"Email Cleaning Error: {str(e)}")
                return "invalid_email@example.com"

        def clean_customer_id(customer_id):
            try:
                corrected_id = re.sub(r"([A-Za-z0-9]+)\1", r"\1", customer_id)
                return corrected_id
            except Exception as e:
                log_error(f"Customer ID Cleaning Error: {str(e)}")
                return customer_id

        customer_data['customer_name'] = customer_data['customer_name'].apply(clean_name)
        customer_data['email'] = customer_data['email'].apply(clean_email)
        customer_data['customer_id'] = customer_data['customer_id'].apply(clean_customer_id)
    except Exception as e:
        log_error(f"Data Transformation Error: {str(e)}")
        raise

    try:
        conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                              'SERVER=DATA_ENGINEER\\SQLEXPRESS;'
                              'DATABASE=sale_db;'
                              'Trusted_Connection=yes;')
        cursor = conn.cursor()
    except Exception as e:
        log_error(f"Database Connection Error: {str(e)}")
        raise

    try:
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name = 'Products' AND xtype = 'U')
        BEGIN
            CREATE TABLE Products (
                product_id VARCHAR(50) PRIMARY KEY,
                product_name VARCHAR(255),
                product_category VARCHAR(255),
                price DECIMAL(10, 2)
            )
        END
        ''')

        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name = 'Transactions' AND xtype = 'U')
        BEGIN
            CREATE TABLE Transactions (
                transaction_id VARCHAR(50) PRIMARY KEY,
                customer_id VARCHAR(50),
                product_id VARCHAR(50),
                quantity INT,
                discount DECIMAL(5, 2),
                date DATETIME,
                region VARCHAR(50),
                total_price DECIMAL(10, 2),
                FOREIGN KEY (product_id) REFERENCES Products(product_id)
            )
        END
        ''')

        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name = 'Customers' AND xtype = 'U')
        BEGIN
            CREATE TABLE Customers (
                customer_id VARCHAR(50) PRIMARY KEY,
                customer_name VARCHAR(1000),
                email VARCHAR(500),
                region VARCHAR(50),
                join_date DATETIME2,
                loyalty_points INT
            )
        END
        ''')

        for _, row in products_data.iterrows():
            try:
                cursor.execute("SELECT COUNT(1) FROM Products WHERE product_id = ?", row['product_id'])
                if cursor.fetchone()[0] > 0:
                    log_error(f"Skipping duplicate product_id: {row['product_id']}")
                    continue

                cursor.execute('''
                INSERT INTO Products (product_id, product_name, product_category, price)
                VALUES (?, ?, ?, ?)''',
                row['product_id'], row['product_name'], row['product_category'], row['price'])
            except Exception as e:
                log_error(f"Error inserting product_id {row['product_id']}: {str(e)}")

        for _, row in sales_data.iterrows():
            try:
                cursor.execute("SELECT COUNT(1) FROM Transactions WHERE transaction_id = ?", row['transaction_id'])
                if cursor.fetchone()[0] > 0:
                    log_error(f"Skipping duplicate transaction_id: {row['transaction_id']}")
                    continue

                cursor.execute('''
                INSERT INTO Transactions (transaction_id, customer_id, product_id, quantity, discount, date, region, total_price)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                row['transaction_id'], row['customer_id'], row['product']['id'], row['quantity'],
                row['discount'], row['date'], row['region'], row['total_price'])
            except Exception as e:
                log_error(f"Error inserting transaction_id {row.get('transaction_id', 'UNKNOWN')}: {str(e)}")

        for _, row in customer_data.iterrows():
            try:
                cursor.execute("SELECT COUNT(1) FROM Customers WHERE customer_id = ?", row['customer_id'])
                if cursor.fetchone()[0] > 0:
                    log_error(f"Skipping duplicate customer_id: {row['customer_id']}")
                    continue

                join_date = row['join_date'] if pd.notna(row['join_date']) else None
                cursor.execute('''
                INSERT INTO Customers (customer_id, customer_name, email, region, join_date, loyalty_points)
                VALUES (?, ?, ?, ?, ?, ?)''',
                row['customer_id'], row['customer_name'], row['email'], row['region'], join_date, row['loyalty_points'])
            except Exception as e:
                log_error(f"Error inserting customer_id {row['customer_id']}: {str(e)}")

        conn.commit()
    except Exception as e:
        log_error(f"Database Operation Error: {str(e)}")
        raise
    finally:
        cursor.close()
        conn.close()

    print("ETL Process Completed Successfully!")

except Exception as e:
    log_error(f"General ETL Error: {str(e)}")
    print("ETL process failed! Check logs for details.")


ETL Process Completed Successfully!
